In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from rac.pred_models import CustomTensorDataset, ACCNet
import numpy as np

In [2]:
transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=True,
                                        download=True, transform=transform)
#trainset.data = trainset.data
#trainset.targets = trainset.targets
X_train = trainset.data
y_train = trainset.targets
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='../datasets/cifar10_original_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1024, num_workers=8)

X_test = testset.data
y_test = testset.targets

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def clustering_from_clustering_solution(clustering_solution):
    num_clusters = np.max(clustering_solution) + 1
    clustering = [[] for _ in range(num_clusters)]
    for i in range(len(clustering_solution)):
        clustering[clustering_solution[i]].append(i)
    return clustering, num_clusters

def sim_matrix_from_clustering(clustering, N):
    pairwise_similarities = -np.ones((N, N))
    for cind in clustering:
        pairwise_similarities[np.ix_(cind, cind)] = 1
    return pairwise_similarities

In [4]:
train_sol = clustering_from_clustering_solution(y_train[:10000])
train_sim_matrix = sim_matrix_from_clustering(train_sol[0], len(y_train[:10000]))

test_sol = clustering_from_clustering_solution(y_test[:10000])
test_sim_matrix = sim_matrix_from_clustering(test_sol[0], len(y_test[:10000]))

In [5]:
train_sim_matrix.shape

(10000, 10000)

In [8]:
def get_pairs(prop_pos, prop_neg, sim_matrix, data):
    N = sim_matrix.shape[0]
    lower_triangle_indices = np.tril_indices(N, -1)
    ind_pos = np.where(sim_matrix[lower_triangle_indices] == 1)[0]
    ind_neg = np.where(sim_matrix[lower_triangle_indices] == -1)[0]
    num_pos = int(len(ind_pos)*prop_pos)
    num_neg = int(len(ind_neg)*prop_neg)
    print("num_pos: ", num_pos)
    print("num_neg: ", num_neg)
    ind_pos = np.random.choice(ind_pos, num_pos)
    ind_neg = np.random.choice(ind_neg, num_neg)
    if num_pos < num_neg:
        indices = np.concatenate([ind_neg, ind_pos])
    else:
        indices = np.concatenate([ind_pos, ind_neg])
    ind1, ind2 = lower_triangle_indices[0][indices], lower_triangle_indices[1][indices]
    x1 = data[ind1]
    x2 = data[ind2]
    y = sim_matrix[ind1, ind2]
    lab1 = np.where(y >= 0)
    lab2 = np.where(y < 0)
    y[lab1] = 1.0
    y[lab2] = 0.0
    return x1, x2, y

In [9]:
x1_train, x2_train, y_train_pairs = get_pairs(0.001, 0.0004, train_sim_matrix, X_train)
x1_test, x2_test, y_test_pairs = get_pairs(0.001, 0.0001, test_sim_matrix, X_test)

num_pos:  4998
num_neg:  17998
num_pos:  4995
num_neg:  4500


In [19]:
cifar_training_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

cifar_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = CustomTensorDataset(x1_train, x2_train, y_train_pairs, train=True, transform=cifar_training_transform)
test_dataset = CustomTensorDataset(x1_test, x2_test, y_test_pairs, transform=cifar_test_transform)  

In [20]:
batch_size = 128
#class_sample_count = [10, 1, 20, 3, 4] # dataset has 10 class-1 samples, 1 class-2 samples, etc.
class_sample_count = np.unique(y_train_pairs, return_counts=True)[1].tolist()
#print(class_sample_count)
weights = 1/torch.Tensor(class_sample_count)
weights = weights[y_train_pairs]
#print(weights)
#sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=1024)
#trainloader = data_utils.DataLoader(train_dataset, batch_size = batch_size, shuffle=True, sampler = sampler)

In [21]:
#for x1, x2, y in train_loader:
    #print(np.unique(y, return_counts=True))

In [22]:
len(train_dataset)

35996

In [23]:
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [24]:
device

device(type='cuda', index=0)

In [25]:
net = ACCNet(base_net="resnet", siamese=True).to(device)

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.0005, momentum=0.9, weight_decay=5e-4)
print("training...")
print(len(train_dataset))
for epoch in range(300):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    step = 0
    for i, data in enumerate(train_loader, 0):
        #print(np.unique(data[2], return_counts=True))
        #print(data[2])
        # get the inputs; data is a list of [inputs, labels]
        x1, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(x1, x2)
        outputs = outputs.reshape((outputs.shape[0]))
        #labels = labels.reshape((labels.shape[0], 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        step += 1
    print("loss: ", running_loss/step)
    step = 0
    running_loss = 0.0

training...
35996
0
loss:  0.6636521120710669
1
loss:  0.602030305286661
2
loss:  0.5832869407089913
3
loss:  0.5674492982020038
4
loss:  0.55176590812405
5
loss:  0.5418877081345423
6
loss:  0.5316940339669275
7
loss:  0.5190746006182687
8
loss:  0.5022265020009212
9
loss:  0.4845648738791193
10
loss:  0.4698902180089805
11
loss:  0.45992514266380685
12
loss:  0.4456329881124204
13
loss:  0.4365959244459083
14
loss:  0.42705169012618344
15
loss:  0.4151002484988555
16
loss:  0.4040943309151104
17
loss:  0.39435345259591853
18
loss:  0.38349228858268425
19
loss:  0.37209667672908164
20
loss:  0.36453477311783206
21
loss:  0.3518229451667971
22
loss:  0.3455842712761126
23
loss:  0.34062823094737527
24
loss:  0.32656256066461653
25
loss:  0.3232939329971339
26
loss:  0.31074385995322745
27
loss:  0.30208116646842015
28
loss:  0.29555488604954366
29
loss:  0.2850794499565056
30
loss:  0.27975909086172973
31
loss:  0.27361189112919715
32
loss:  0.26116069581189805
33
loss:  0.256998972440

In [17]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
preds = []
with torch.no_grad():
    for data in test_loader:
        x1, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        # calculate outputs by running images through the network
        outputs = net(x1, x2)
        pred = nn.Sigmoid()(outputs)
        #print(pred)
        pred = pred.cpu().numpy()
        pred = pred.reshape(pred.shape[0])
        pred[pred >= 0.5] = 1
        pred[pred < 0.5] = 0
        #print(torch.max(pred.data, 1)[1])
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(pred.data, 1)
        preds.extend(pred.tolist())


In [18]:
len(preds)

9495

In [19]:
np.unique(preds, return_counts=True)

(array([0., 1.]), array([6497, 2998]))

In [20]:
#preds

In [24]:
len(y_test_pairs)


9495

In [21]:
y_test_pairs

array([1., 1., 1., ..., 0., 0., 0.])

In [22]:
np.unique(y_test_pairs, return_counts=True)

(array([0., 1.]), array([4500, 4995]))

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test_pairs, preds))

              precision    recall  f1-score   support

         0.0       0.62      0.89      0.73      4500
         1.0       0.83      0.50      0.62      4995

    accuracy                           0.68      9495
   macro avg       0.72      0.69      0.68      9495
weighted avg       0.73      0.68      0.67      9495

